In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [ ]:
pip install kaggle

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kushagra3204/wheat-plant-diseases")

print("Path to dataset files:", path)

100%|██████████| 6.09G/6.09G [04:49<00:00, 22.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/kushagra3204/wheat-plant-diseases/versions/6


In [3]:
import os

base_path = "/root/.cache/kagglehub/datasets/kushagra3204/wheat-plant-diseases/versions/6/data"

for root, dirs, files in os.walk(base_path):
    level = root.replace(base_path, '').count(os.sep)
    indent = ' ' * 4 * level
    folder_name = os.path.basename(root)
    file_count = len([f for f in files if not f.startswith('.')])
    print(f"{indent}{folder_name}/ ({file_count} files)")


data/ (0 files)
    valid/ (0 files)
        yellow_rust_valid/ (20 files)
        mite_valid/ (20 files)
        blast_test_valid/ (20 files)
        leaf_blight_valid/ (20 files)
        tan_spot_valid/ (20 files)
        brown_rust_valid/ (20 files)
        aphid_valid/ (20 files)
        common_root_rot_valid/ (20 files)
        mildew_valid/ (20 files)
        smut_valid/ (20 files)
        black_rust_valid/ (20 files)
        septoria_valid/ (20 files)
        fusarium_head_blight_valid/ (20 files)
        stem_fly_valid/ (20 files)
        healthy_valid/ (20 files)
    test/ (0 files)
        septoria_test/ (50 files)
        blast_test/ (50 files)
        mite_test/ (50 files)
        tan_spot_test/ (50 files)
        yellow_rust_test/ (50 files)
        healthy_test/ (50 files)
        brown_rust_test/ (50 files)
        stem_fly_test/ (50 files)
        leaf_blight_test/ (50 files)
        black_rust_test/ (50 files)
        mildew_test/ (50 files)
        smut_test/ (50 file

In [ ]:
!pip install tensorflow

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Function to rename validation and test folders to match training class names
def clean_folder_names(base_subpath):
    original_path = os.path.join(base_path, base_subpath)
    for folder in os.listdir(original_path):
        folder_path = os.path.join(original_path, folder)
        if os.path.isdir(folder_path):
            # Remove suffix like '_test' or '_valid', replace underscores with spaces and capitalize
            cleaned = folder.replace('_test', '').replace('_valid', '').replace('_', ' ').title()
            new_path = os.path.join(original_path, cleaned)
            if folder_path != new_path:
                os.rename(folder_path, new_path)


In [5]:

# Clean 'valid' and 'test' folder names
clean_folder_names('valid')
clean_folder_names('test')

# Define image size and batch
img_size = (64, 64)
batch_size = 32


In [6]:

# Setup image data generators
train_path = os.path.join(base_path, "train")
valid_path = os.path.join(base_path, "valid")
test_path = os.path.join(base_path, "test")


In [7]:

train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(train_path, target_size=img_size, batch_size=batch_size, class_mode='categorical')
valid_data = valid_datagen.flow_from_directory(valid_path, target_size=img_size, batch_size=batch_size, class_mode='categorical')
test_data = test_datagen.flow_from_directory(test_path, target_size=img_size, batch_size=batch_size, class_mode='categorical', shuffle=False)



Found 13104 images belonging to 15 classes.
Found 300 images belonging to 15 classes.
Found 750 images belonging to 15 classes.


In [8]:

# Lightweight CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(train_data.num_classes, activation='softmax')
])



/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_data, validation_data=valid_data, epochs=30)

Epoch 1/30
410/410 ━━━━━━━━━━━━━━━━━━━━ 207s 498ms/step - accuracy: 0.5469 - loss: 1.4471 - val_accuracy: 0.4633 - val_loss: 1.8423
Epoch 2/30
410/410 ━━━━━━━━━━━━━━━━━━━━ 192s 467ms/step - accuracy: 0.6197 - loss: 1.1825 - val_accuracy: 0.5300 - val_loss: 1.6873
Epoch 3/30
410/410 ━━━━━━━━━━━━━━━━━━━━ 196s 480ms/step - accuracy: 0.6856 - loss: 0.9645 - val_accuracy: 0.6200 - val_loss: 1.5298
Epoch 4/30
410/410 ━━━━━━━━━━━━━━━━━━━━ 208s 507ms/step - accuracy: 0.7453 - loss: 0.7671 - val_accuracy: 0.6900 - val_loss: 1.5394
Epoch 5/30
410/410 ━━━━━━━━━━━━━━━━━━━━ 192s 468ms/step - accuracy: 0.7973 - loss: 0.6124 - val_accuracy: 0.6933 - val_loss: 1.3879
Epoch 6/30
410/410 ━━━━━━━━━━━━━━━━━━━━ 195s 478ms/step - accuracy: 0.8489 - loss: 0.4684 - val_accuracy: 0.8133 - val_loss: 1.1665
Epoch 7/30
410/410 ━━━━━━━━━━━━━━━━━━━━ 191s 467ms/step - accuracy: 0.8793 - loss: 0.3643 - val_accuracy: 0.8433 - val_loss: 1.2229
Epoch 8/30
410/410 ━━━━━━━━━━━━━━━━━━━━ 207s 503ms/step - accuracy: 0.9065 -

In [ ]:
import tensorflow as tf
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

model = tf.keras.models.load_model('wheat_disease_cnn.h5')

In [9]:
# Evaluate the trained model on the test set
loss, accuracy = model.evaluate(test_data)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.2%}")


ValueError: You must call `compile()` before using the model.

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Get predictions
predictions = model.predict(test_data)
y_pred = np.argmax(predictions, axis=1)
y_true = test_data.classes

# Get class labels
class_labels = list(test_data.class_indices.keys())

# Generate classification report
report = classification_report(y_true, y_pred, target_names=class_labels, output_dict=True)

# Display results like your screenshot
print(f"{'Class':<25} {'Accuracy (%)':<15} {'Correct':<10} {'Total'}")
for i, label in enumerate(class_labels):
    correct = int(report[label]['support'] * report[label]['recall'])
    total = report[label]['support']
    accuracy = correct / total * 100
    print(f"{label:<25} {accuracy:<15.1f} {correct:<10} {total}")


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


24/24 ━━━━━━━━━━━━━━━━━━━━ 6s 239ms/step
Class                     Accuracy (%)    Correct    Total
Aphid                     100.0           50         50.0
Black Rust                84.0            42         50.0
Blast                     100.0           50         50.0
Brown Rust                96.0            48         50.0
Common Root Rot           100.0           50         50.0
Fusarium Head Blight      98.0            49         50.0
Healthy                   8.0             4          50.0
Leaf Blight               98.0            49         50.0
Mildew                    94.0            47         50.0
Mite                      96.0            48         50.0
Septoria                  92.0            46         50.0
Smut                      100.0           50         50.0
Stem Fly                  100.0           50         50.0
Tan Spot                  92.0            46         50.0
Yellow Rust               96.0            48         50.0


In [ ]:
import tensorflow as tf
from sklearn.metrics import precision_score, recall_score, f1_score

model = tf.keras.models.load_model('wheat_disease_cnn.h5')

y_true = test_data.classes  # Directly get the ground truth labels
print(f"True labels shape: {y_true.shape}")

test_data.reset()

# Calculate the number of steps per epoch for the prediction
steps = len(test_data)

# Predict the test data with the correct number of steps
print("Starting prediction...")
y_pred = model.predict(test_data, steps=steps, verbose=1)
print("Prediction completed.")

y_pred_classes = y_pred.argmax(axis=1)

print(f"Predicted labels length: {len(y_pred_classes)}")
print(f"True labels length: {len(y_true)}")

# Calculate metrics
precision = precision_score(y_true, y_pred_classes, average='macro')
recall = recall_score(y_true, y_pred_classes, average='macro')
f1 = f1_score(y_true, y_pred_classes, average='macro')

print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")

True labels shape: (750,)
Starting prediction...
24/24 ━━━━━━━━━━━━━━━━━━━━ 6s 226ms/step
Prediction completed.
Predicted labels length: 750
True labels length: 750
Precision: 0.917
Recall: 0.903
F1 Score: 0.885
